In [10]:
import numpy as np
import torch
from torch import nn

path = "../data/ex1.npy"

with open(path, 'rb') as f:
    P = np.load(f)
    Ns = np.load(f)

In [40]:
class SUCPA(nn.Module):
    
    def __init__(self, num_classes, steps=10):
        super().__init__()
        self.num_classes = num_classes
        self.steps = steps
        self.beta = nn.Parameter(torch.zeros(num_classes), requires_grad=False)
        self.jacobian = torch.zeros(num_classes, num_classes)
        self.beta_history = None
        self.jacobian_history = None

    def fit(self, logits, class_samples):
        beta_history = [self.beta.data]
        jacobian_history = [self.jacobian]
        for i in range(self.steps):
            log_den = torch.logsumexp(logits + self.beta, dim=1)
            log_sum_all = torch.logsumexp(logits.T - log_den, dim=1)
            self.beta.data = -log_sum_all + torch.log(class_samples)
            beta_history.append(self.beta.data)
            
            probs = torch.softmax(logits, dim=1)
            exp_beta = torch.exp(self.beta)
            den = probs @ exp_beta.unsqueeze(1)
            probs_den = probs / den
            self.jacobian = (probs_den.T @ torch.softmax(logits + self.beta, dim=1)) / probs_den.sum(dim=0).unsqueeze(1)
            jacobian_history.append(self.jacobian)
        self.beta_history = torch.stack(beta_history)
        self.jacobian_history = torch.stack(jacobian_history)
        return self
    
    def calibrate(self, logits):
        return logits + self.beta

model = SUCPA(2, steps=10)
model.fit(torch.log(torch.from_numpy(P))-torch.rand(P.shape[0],1), torch.from_numpy(Ns))
print(model.beta_history.numpy())
print(np.linalg.eig(model.jacobian_history)[0])

[[ 0.          0.        ]
 [ 0.14142656 -0.10445261]
 [ 0.15051031 -0.11197329]
 [ 0.15112734 -0.11248779]
 [ 0.15116978 -0.11252308]
 [ 0.15117264 -0.11252499]
 [ 0.15117311 -0.11252499]
 [ 0.15117311 -0.11252451]
 [ 0.15117359 -0.11252451]
 [ 0.15117359 -0.11252403]
 [ 0.15117407 -0.11252403]]
[[0.         0.        ]
 [0.0681585  1.0000001 ]
 [0.06820678 1.0000001 ]
 [0.06820999 1.0000001 ]
 [0.06821015 1.        ]
 [0.06821031 1.        ]
 [0.06821037 1.        ]
 [0.06821038 1.0000002 ]
 [0.06821037 1.        ]
 [0.0682103  1.        ]
 [0.06821026 1.        ]]


In [4]:
[n,C] = P.shape

In [5]:
def UCPA(P, Ns, beta):
    D = 1.0 / (P @ np.exp(beta))
    S = P.T@D
    next_beta = -np.log(S/Ns)
    J = ((((P*np.exp(beta)).T*(D**2))@ P)/S).T
    return next_beta, J
    

In [6]:
pasos = 10
BETA = np.zeros((pasos+1,C))
JACOBOS = np.zeros((pasos,C,C))
for k in range(pasos):
    BETA[k+1,:],JACOBOS[k,:,:] = UCPA(P,Ns,BETA[k,:])

In [29]:
model.beta_history.numpy()

array([[ 0.        ,  0.        ],
       [ 0.14142656, -0.10445261],
       [ 0.15051031, -0.11197281],
       [ 0.15112734, -0.11248779],
       [ 0.15116978, -0.11252308],
       [ 0.15117264, -0.11252499],
       [ 0.15117311, -0.11252499],
       [ 0.15117311, -0.11252451],
       [ 0.15117359, -0.11252451],
       [ 0.15117359, -0.11252403],
       [ 0.15117407, -0.11252403]], dtype=float32)

In [5]:
BETA # Actualizacion de los parametros iteracion a iteracion

array([[ 0.        ,  0.        ],
       [ 0.14142691, -0.10445262],
       [ 0.15051016, -0.1119732 ],
       [ 0.15112731, -0.11248815],
       [ 0.1511694 , -0.11252328],
       [ 0.15117227, -0.11252568],
       [ 0.15117246, -0.11252584],
       [ 0.15117248, -0.11252585],
       [ 0.15117248, -0.11252585],
       [ 0.15117248, -0.11252585],
       [ 0.15117248, -0.11252585]])

In [8]:
np.linalg.eig(JACOBOS)[0] #Actualizacion de los autovalores del Jacobiano iteracion a iteracion

array([[0.06665432, 1.        ],
       [0.06815863, 1.        ],
       [0.06820681, 1.        ],
       [0.06820985, 1.        ],
       [0.06821005, 1.        ],
       [0.06821006, 1.        ],
       [0.06821007, 1.        ],
       [0.06821007, 1.        ],
       [0.06821007, 1.        ],
       [0.06821007, 1.        ]])

In [6]:
np.linalg.eig(JACOBOS)[0] #Actualizacion de los autovalores del Jacobiano iteracion a iteracion

array([[0.06665432, 1.        ],
       [0.06815863, 1.        ],
       [0.06820681, 1.        ],
       [0.06820985, 1.        ],
       [0.06821005, 1.        ],
       [0.06821006, 1.        ],
       [0.06821007, 1.        ],
       [0.06821007, 1.        ],
       [0.06821007, 1.        ],
       [0.06821007, 1.        ]])